---

### Step 3 - Develop Baseline Demand Prediction Model

With Steps 1 and 2 complete, we are ready to create a baseline model for for predicting demand. Generally, we create baseline model using very basic techniques like mean prediction and then we try more complex solutions to improve the results that we got from the baseline model.

---
In this Notebook:
 - We will first of all predict the target using the mean of previous data. For, a particular store, how many average number of units were sold of a particular product.
 - We will create a Regression based model and Tree based model.
 - We will use Simple Moving Average (a very basic Time Series Model).
 - We will use data to train model with different time periods and check whether there is a recency factor in our data.

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import datasets.
salesdf = pd.read_csv('data/salesdf.csv')
productsdf = pd.read_csv('data/productsdf.csv')
storesdf = pd.read_csv('data/storesdf.csv')

In [3]:
salesdf.head()

WEEK_END_DATE  STORE_NUM         UPC  PRICE  BASE_PRICE  FEATURE  DISPLAY  \
0     13-Jan-16        367  1111009477   1.39        1.57        0        0   
1     13-Jan-16        367  1111009497   1.39        1.39        0        0   
2     13-Jan-16        367  1111085319   1.88        1.88        0        0   
3     13-Jan-16        367  1111085345   1.88        1.88        0        0   
4     13-Jan-16        367  1111085350   1.98        1.98        0        0   

   UNITS  
0     13  
1     20  
2     14  
3     29  
4     35

In [4]:
productsdf.head()

UPC  MANUFACTURER_1  MANUFACTURER_2  MANUFACTURER_3  MANUFACTURER_4  \
0  1111009477               1               0               0               0   
1  1111009497               1               0               0               0   
2  1111009507               1               0               0               0   
3  1111038078               1               0               0               0   
4  1111038080               1               0               0               0   

   MANUFACTURER_5  MANUFACTURER_6  MANUFACTURER_7  MANUFACTURER_8  \
0               0               0               0               0   
1               0               0               0               0   
2               0               0               0               0   
3               0               0               0               0   
4               0               0               0               0   

   MANUFACTURER_9  ...  CATEGORY_3  CATEGORY_4  SUB_CATEGORY_1  \
0               0  ...           0           0               1   
1               0  ...           0           0               1   
2               0  ...           0           0               1   
3               0  ...           0           0               0   
4               0  ...           0           0               0   

   SUB_CATEGORY_2  SUB_CATEGORY_3  SUB_CATEGORY_4  SUB_CATEGORY_5  \
0               0               0               0               0   
1               0               0               0               0   
2               0               0               0               0   
3               1               0               0               0   
4               1               0               0               0   

   SUB_CATEGORY_6  SUB_CATEGORY_7  PRODUCT_SIZE  
0               0               0             2  
1               0               0             2  
2               0               0             2  
3               0               0             1  
4               0               0             1  

[5 rows x 22 columns]

In [5]:
storesdf.head()

STORE_ID  ADDRESS_STATE_PROV_CODE_1  ADDRESS_STATE_PROV_CODE_2  \
0       367                          1                          0   
1       389                          1                          0   
2       613                          0                          1   
3       623                          0                          1   
4      2277                          0                          0   

   ADDRESS_STATE_PROV_CODE_3  ADDRESS_STATE_PROV_CODE_4  MSA_CODE_1  \
0                          0                          0           1   
1                          0                          0           1   
2                          0                          0           0   
3                          0                          0           0   
4                          1                          0           1   

   MSA_CODE_2  MSA_CODE_3  MSA_CODE_4  MSA_CODE_5  MSA_CODE_6  MSA_CODE_7  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           1           0           0           0           0           0   
3           0           1           0           0           0           0   
4           0           0           0           0           0           0   

   MSA_CODE_8  MSA_CODE_9  SEG_VALUE_NAME  SALES_AREA_SIZE_NUM  \
0           0           0               1                24721   
1           0           0               2                46073   
2           0           0               2                64926   
3           0           0               2                46930   
4           0           0               3                81958   

   AVG_WEEKLY_BASKETS  
0               12707  
1               24767  
2               29386  
3               36741  
4               54053

In [6]:
# Merge datasets.
basemodel = salesdf.merge(productsdf, how='left', on= 'UPC')
basemodel = basemodel.merge(storesdf, how= 'left', left_on= 'STORE_NUM', right_on= 'STORE_ID')

In [7]:
basemodel = basemodel.drop(columns=['STORE_ID'])

In [9]:
# Check for null values in dataset.
basemodel.isna().sum().sum()

0

In [10]:
basemodel.loc[0]

WEEK_END_DATE                 13-Jan-16
STORE_NUM                           367
UPC                          1111009477
PRICE                              1.39
BASE_PRICE                         1.57
FEATURE                               0
DISPLAY                               0
UNITS                                13
MANUFACTURER_1                        1
MANUFACTURER_2                        0
MANUFACTURER_3                        0
MANUFACTURER_4                        0
MANUFACTURER_5                        0
MANUFACTURER_6                        0
MANUFACTURER_7                        0
MANUFACTURER_8                        0
MANUFACTURER_9                        0
CATEGORY_1                            1
CATEGORY_2                            0
CATEGORY_3                            0
CATEGORY_4                            0
SUB_CATEGORY_1                        1
SUB_CATEGORY_2                        0
SUB_CATEGORY_3                        0
SUB_CATEGORY_4                        0


In [11]:
# Convert the WEEK_END_DATE to datetime format.
basemodel.WEEK_END_DATE = pd.to_datetime(basemodel.WEEK_END_DATE)

---

- Store unique `WEEK_END_DATE` values in a list to more easily split the data based on datetime, which is required to split data into train and validation datasets.

- Create a one-week gap between the train and validation datasets, with the train set starting from the very begining of the dates in our model.

---

In [12]:
# Store unique dates in a list.
weeks = basemodel.WEEK_END_DATE.unique()

In [13]:
# Define function that will return a dictionary which contains keys. There will be 1 week gap between train and validation datasets.

def get_train_validation_set(number=1):
    validation_sets = []
    for n in range(number):
        x = {}
        
        x['validation_set'] = weeks[len(weeks)-n-1]
        x['train_set_end_date'] = weeks[len(weeks)-n-3]
        validation_sets.append(x)
        
    return validation_sets       

In [14]:
# Create baseline model and test on 5 different datasets in order to check the consistency of Baseline RMSLE scores across multiple subsets of data. 
validation_sets = get_train_validation_set(number=5)

In [15]:
# Examine dictionary created.
validation_sets

[{'validation_set': numpy.datetime64('2018-09-26T00:00:00.000000000'),
  'train_set_end_date': numpy.datetime64('2018-09-12T00:00:00.000000000')},
 {'validation_set': numpy.datetime64('2018-09-19T00:00:00.000000000'),
  'train_set_end_date': numpy.datetime64('2018-09-05T00:00:00.000000000')},
 {'validation_set': numpy.datetime64('2018-09-12T00:00:00.000000000'),
  'train_set_end_date': numpy.datetime64('2018-08-29T00:00:00.000000000')},
 {'validation_set': numpy.datetime64('2018-09-05T00:00:00.000000000'),
  'train_set_end_date': numpy.datetime64('2018-08-22T00:00:00.000000000')},
 {'validation_set': numpy.datetime64('2018-08-29T00:00:00.000000000'),
  'train_set_end_date': numpy.datetime64('2018-08-15T00:00:00.000000000')}]

In [16]:
# Use dictionary and store train and validation sets as a list of tuples.

data_set = []

for data in validation_sets:
    
    traindf = basemodel[basemodel.WEEK_END_DATE <= data['train_set_end_date']]
    validatedf = basemodel[basemodel.WEEK_END_DATE == data['validation_set']]
    
    data_set.append((traindf, validatedf))

---

### Baseline #1: Mean Prediction


Now, we will create our first baseline model, `MEAN PREDICTION`. We will use the past data to and take average on a group of `STORE_NUM` and `UPC` and use this to predict on the validaion set.


#### `Evaluation Metric: ` Root Mean Squared Log Error (RMSLE)
---

In [17]:
# Define function to get the RMSLE.
from sklearn.metrics import mean_squared_log_error as msle

def get_msle(true, predicted) :
    return np.sqrt(msle( true, predicted))

In [18]:
train_rmsle = []
valid_rmsle = []

for i, data in enumerate(data_set):
    
    # get the train and validation set
    train, valid = data
    
    # get the mean prediction dataframe by using a groupby on STORE_NUM and UPC
    mean_prediction = train.groupby(['STORE_NUM', 'UPC'])['UNITS'].mean().reset_index()
    
    # left join the train and validation set with the mean prediction.
    train = train.merge(mean_prediction, how='left', on=['STORE_NUM', 'UPC'])
    valid = valid.merge(mean_prediction, how='left', on=['STORE_NUM', 'UPC'])
    
    # In the updated dataframe after the left join, 
    # column UNITS_x is the original value of the target variable
    # column UNITS_y is the predicted value of the target variable
    
    
    # get the rmsle on train and validation set
    t_rmsle = get_msle(train.UNITS_x, train.UNITS_y)
    v_rmsle = get_msle(valid.UNITS_x, valid.UNITS_y)
    train_rmsle.append(t_rmsle)
    valid_rmsle.append(v_rmsle)
    
    print('RMSLE ON TRAINING SET: ',i+1, ': ', t_rmsle)
    print('RMSLE ON VALIDATION SET: ',i+1, ': ',v_rmsle)
    print('=====================================================================')
    
# get the mean RMSLE on train and validation set.     
print('Mean RMSLE on Train: ', np.mean(train_rmsle))
print('Mean RMSLE on Valid: ', np.mean(valid_rmsle))

RMSLE ON TRAINING SET:  1 :  0.5902468460088598
RMSLE ON VALIDATION SET:  1 :  0.5887816704436897
RMSLE ON TRAINING SET:  2 :  0.591251579931832
RMSLE ON VALIDATION SET:  2 :  0.6263156060802706
RMSLE ON TRAINING SET:  3 :  0.5917841764867795
RMSLE ON VALIDATION SET:  3 :  0.47837118281730495
RMSLE ON TRAINING SET:  4 :  0.5914233373769653
RMSLE ON VALIDATION SET:  4 :  0.5811759211472836
RMSLE ON TRAINING SET:  5 :  0.5916269229162222
RMSLE ON VALIDATION SET:  5 :  0.718159952727328
Mean RMSLE on Train:  0.5912665725441318
Mean RMSLE on Valid:  0.5985608666431753


#### **This result is just ok. We need to try more models to see how it compares.**

---

### Baseline #2 - Simple Moving Average

- The predicted value will be the average number of UNITS sold in last 8 weeks from a particular store for a particular product.
- As, we have one week gap between the train and validation set. So the last week prediction values in the training set will be used as the prediction values for the validation set.

---

In [19]:
def get_sma(i, train, valid, no_of_weeks=2):
    
    # create a copy of train and validation set
    train_copy = train.copy()
    valid_copy = valid.copy()
    
    # group the data by STORE_NUM and UPC and use rolling and mean function to calculate the moving average.
    data_copy = train_copy.groupby(['STORE_NUM','UPC'])['UNITS'].rolling(no_of_weeks).mean().reset_index().set_index('level_2')
    
    # add the moving average column to the train data
    train_copy['moving_average'] = data_copy['UNITS']
    
    # the last prediction on train set will be used as prediciton on validation set.
    # calculate the last_average dataframe by groupby using last function.
    last_average = train_copy.groupby(['STORE_NUM', 'UPC'])['moving_average'].last().reset_index()
    
    train_copy = train_copy[['WEEK_END_DATE', 'STORE_NUM', 'UPC', 'UNITS', 'moving_average']]
    valid_copy = valid_copy[['WEEK_END_DATE','STORE_NUM', 'UPC', 'UNITS']]
    
    # drop the null values in the dataframe
    train_copy.dropna(inplace=True)
    # merge the validation data with the last_average by left join
    valid_copy = valid_copy.merge(last_average, how= 'left', on= ['STORE_NUM', 'UPC'])
    
    # calculate the rmsle on train and validation data
    t_rmsle = get_msle(train_copy['UNITS'], train_copy['moving_average'])
    v_rmsle = get_msle(valid_copy['UNITS'], valid_copy['moving_average'])
       
        
    print('RMSLE ON TRAINING SET: ',i+1, ': ', t_rmsle)
    print('RMSLE ON VALIDATION SET: ',i+1, ': ',v_rmsle)
    print('=====================================================================')
    
    return t_rmsle, v_rmsle
    

In [20]:
train_rmsle_ma = []
valid_rmsle_ma = []

for i, data in enumerate(data_set):
    train, valid = data

    t_rmsle, v_rmsle = get_sma(i,train, valid, no_of_weeks=8)
    train_rmsle_ma.append(t_rmsle)
    valid_rmsle_ma.append(v_rmsle)
    
print('Mean RMSLE on Train: ', np.mean(train_rmsle_ma))
print('Mean RMSLE on Valid: ', np.mean(valid_rmsle_ma))    

RMSLE ON TRAINING SET:  1 :  0.532290520757075
RMSLE ON VALIDATION SET:  1 :  0.5469206496913668
RMSLE ON TRAINING SET:  2 :  0.5332313430963387
RMSLE ON VALIDATION SET:  2 :  0.6421015703332319
RMSLE ON TRAINING SET:  3 :  0.5335581839226429
RMSLE ON VALIDATION SET:  3 :  0.46149085909723564
RMSLE ON TRAINING SET:  4 :  0.5324765840327685
RMSLE ON VALIDATION SET:  4 :  0.5878031103068386
RMSLE ON TRAINING SET:  5 :  0.5318645623862213
RMSLE ON VALIDATION SET:  5 :  0.7558881602487321
Mean RMSLE on Train:  0.5326842388390093
Mean RMSLE on Valid:  0.598840869935481


#### **This model didn't perform much better than the Mean Prediction model.**

---
### Linear Regresssion

- Now, we will try one Regression Based Model and see how it performs on our dataset. We will use the same 5 validation sets and compare the results. 
- Keys `WEEK_END_DATE`, `STORE_NUM` and `UPC` will be dropped.

---

In [21]:
from sklearn.linear_model import LinearRegression
train_rmsle_lr = []
valid_rmsle_lr = []

for i, data in enumerate(data_set):
    
    train, valid = data
    
    # drop the columns that are not required, separate the target and independent features
    train_x = train.drop(columns=['WEEK_END_DATE', 'STORE_NUM', 'UPC', 'UNITS'])
    train_y = train['UNITS']
    
    valid_x = valid.drop(columns=['WEEK_END_DATE', 'STORE_NUM', 'UPC', 'UNITS'])
    valid_y = valid['UNITS']
    
    # create an Object of the Linear Regression model
    model_LR = LinearRegression(normalize=True)
    # fit the model with  the training data
    model_LR.fit(train_x, train_y)
    
    # predict on the training data 
    # the model can predict some negative values also and RMSLE only supports positive values.
    # So, we will use the clip function. It will convert all the negative predicted values to 0.
    predict_train = model_LR.predict(train_x).clip(min=0)
    predict_valid = model_LR.predict(valid_x).clip(min=0)
    
    # get the rmsle on the training and validation data.
    t_rmsle = get_msle(train_y, predict_train)
    v_rmsle = get_msle(valid_y, predict_valid)
    train_rmsle_lr.append(t_rmsle)
    valid_rmsle_lr.append(v_rmsle)
    
    print('RMSLE ON TRAINING SET: ',i+1, ': ', t_rmsle)
    print('RMSLE ON VALIDATION SET: ',i+1, ': ',v_rmsle)
    print('=====================================================================')
    
    
    
print('Mean RMSLE on Train: ', np.mean(train_rmsle_lr))
print('Mean RMSLE on Valid: ', np.mean(valid_rmsle_lr))    
    

RMSLE ON TRAINING SET:  1 :  0.9856560459029289
RMSLE ON VALIDATION SET:  1 :  0.9466737000535473
RMSLE ON TRAINING SET:  2 :  0.9854516120683233
RMSLE ON VALIDATION SET:  2 :  0.9276683506412722
RMSLE ON TRAINING SET:  3 :  0.9904294711153601
RMSLE ON VALIDATION SET:  3 :  0.9593632305560731
RMSLE ON TRAINING SET:  4 :  0.978111045337829
RMSLE ON VALIDATION SET:  4 :  0.9119993845391732
RMSLE ON TRAINING SET:  5 :  0.988499547504901
RMSLE ON VALIDATION SET:  5 :  0.9615990929915409
Mean RMSLE on Train:  0.9856295443858685
Mean RMSLE on Valid:  0.9414607517563214


### **Linear regression performed much worse than the other two baseline models.

---

### Baseline #4 - Decision Tree

- Tree Based Model. We will use the same 5 validation sets and compare the results.
- Again, keys `WEEK_END_DATE`, `STORE_NUM` and `UPC` will be dropped.

---

In [22]:
from sklearn.tree import DecisionTreeRegressor

train_rmsle_dtr = []
valid_rmsle_dtr = []

for i, data in enumerate(data_set):
    
    # get the train and validation set
    train, valid = data
    
    # drop the columns that are not required, separate the target and independent features    
    train_x = train.drop(columns=['WEEK_END_DATE', 'STORE_NUM', 'UPC', 'UNITS'])
    train_y = train['UNITS']
    
    valid_x = valid.drop(columns=['WEEK_END_DATE', 'STORE_NUM', 'UPC', 'UNITS'])
    valid_y = valid['UNITS']
    
    # create an Object of DecisionTree Regressor
    model_DTR = DecisionTreeRegressor()
    # fit the model with the training data
    model_DTR.fit(train_x, train_y)
    
    # predict the target and set the minimum value of the predicted target variable to be 0
    predict_train = model_DTR.predict(train_x).clip(min=0)
    predict_valid = model_DTR.predict(valid_x).clip(min=0)
    
    # get the rmsle on train and validation set.
    t_rmsle = get_msle(train_y, predict_train)
    v_rmsle = get_msle(valid_y, predict_valid)
    
    train_rmsle_dtr.append(t_rmsle)
    valid_rmsle_dtr.append(v_rmsle)
    
    print('RMSLE ON TRAINING SET: ',i+1, ': ', t_rmsle)
    print('RMSLE ON VALIDATION SET: ',i+1, ': ',v_rmsle)
    print('=====================================================================')
    
    
    
print('Mean RMSLE on Train: ', np.mean(train_rmsle_dtr))
print('Mean RMSLE on Valid: ', np.mean(valid_rmsle_dtr))    

RMSLE ON TRAINING SET:  1 :  0.3509603160009687
RMSLE ON VALIDATION SET:  1 :  0.45164333921186794
RMSLE ON TRAINING SET:  2 :  0.35068405865135405
RMSLE ON VALIDATION SET:  2 :  0.4818260918049475
RMSLE ON TRAINING SET:  3 :  0.3502097858790728
RMSLE ON VALIDATION SET:  3 :  0.45577650435346795
RMSLE ON TRAINING SET:  4 :  0.34962464321046194
RMSLE ON VALIDATION SET:  4 :  0.5072881470926939
RMSLE ON TRAINING SET:  5 :  0.3496036689909416
RMSLE ON VALIDATION SET:  5 :  0.5887580530350959
Mean RMSLE on Train:  0.35021649454655984
Mean RMSLE on Valid:  0.4970584270996146


### **Decision Tree performed much better than any of the previous linear baseline models.

---

### Baseline #5 - RandomForest

Another Decision Tree based model. 

---

In [23]:
from sklearn.ensemble import RandomForestRegressor

train_rmsle_dtr = []
valid_rmsle_dtr = []

for i, data in enumerate(data_set):
    
    # get the train and vaidation set
    train, valid = data
    
    # drop the columns that are not required, separate the target and independent features     
    train_x = train.drop(columns=['WEEK_END_DATE', 'STORE_NUM', 'UPC', 'UNITS'])
    train_y = train['UNITS']
    
    valid_x = valid.drop(columns=['WEEK_END_DATE', 'STORE_NUM', 'UPC', 'UNITS'])
    valid_y = valid['UNITS']
    
    # create an object of the Random Forest Regressor
    model_DTR = RandomForestRegressor(random_state=0)
    
    # fit the model with the training data
    model_DTR.fit(train_x, train_y)
    
    # predict the target and set the minimum value of the predicted target variable to be 0
    predict_train = model_DTR.predict(train_x).clip(min=0)
    predict_valid = model_DTR.predict(valid_x).clip(min=0)
    
    # get the rmsle on train and validate
    t_rmsle = get_msle(train_y, predict_train)
    v_rmsle = get_msle(valid_y, predict_valid)
    
    train_rmsle_dtr.append(t_rmsle)
    valid_rmsle_dtr.append(v_rmsle)
    
    print('RMSLE ON TRAINING SET: ',i+1, ': ', t_rmsle)
    print('RMSLE ON VALIDATION SET: ',i+1, ': ',v_rmsle)
    print('=====================================================================')
    
    
print('Mean RMSLE on Train: ', np.mean(train_rmsle_dtr))
print('Mean RMSLE on Valid: ', np.mean(valid_rmsle_dtr))    
    

RMSLE ON TRAINING SET:  1 :  0.3682865332962354
RMSLE ON VALIDATION SET:  1 :  0.42012300710245326
RMSLE ON TRAINING SET:  2 :  0.3681797274754799
RMSLE ON VALIDATION SET:  2 :  0.4451762845496011
RMSLE ON TRAINING SET:  3 :  0.3677729846864488
RMSLE ON VALIDATION SET:  3 :  0.4337344871926344
RMSLE ON TRAINING SET:  4 :  0.36738188410500855
RMSLE ON VALIDATION SET:  4 :  0.48841969624697673
RMSLE ON TRAINING SET:  5 :  0.367345916149398
RMSLE ON VALIDATION SET:  5 :  0.557354356567105
Mean RMSLE on Train:  0.3677934091425141
Mean RMSLE on Valid:  0.4689615663317541


#### **The RandomForest model seems to be a slight improvement over the Decision Tree model. This will be the baseline model used for the remainder of the project.**